In [2]:
import numpy as np
from enum import Enum
from typing import Dict, Tuple

In [3]:
# helpful enums

# object types - bottle, can, or glass
class ObjectType(Enum):
    BOTTLE = 1
    CAN = 2
    GLASS = 3

# outcomes
class OutcomeType(Enum):
    SP_SUCCESS = 0 # robot succeeds
    SP_FAIL = 1 # robot fails
    IT = 2 # human intervenes

TRUST_LEVELS = [1,2,3,4,5,6,7]

In [4]:
# reward table

REWARD_TABLE: Dict[Tuple[ObjectType, OutcomeType], float] = {
    (ObjectType.BOTTLE, OutcomeType.SP_SUCCESS): 1.0,
    (ObjectType.BOTTLE, OutcomeType.SP_FAIL):    0.0,
    (ObjectType.BOTTLE, OutcomeType.IT):         0.0,

    (ObjectType.CAN, OutcomeType.SP_SUCCESS):    2.0,
    (ObjectType.CAN, OutcomeType.SP_FAIL):      -4.0,
    (ObjectType.CAN, OutcomeType.IT):            0.0,

    (ObjectType.GLASS, OutcomeType.SP_SUCCESS):  3.0,
    (ObjectType.GLASS, OutcomeType.SP_FAIL):    -12.0,
    (ObjectType.GLASS, OutcomeType.IT):          0.0,
}

In [5]:
# adding failure rate
ROBOT_FAIL_RATE = {
    ObjectType.BOTTLE: 0.00,
    ObjectType.CAN:    0.00,
    ObjectType.GLASS:  0.25,
}


In [6]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

class HumanDecisionModel:

  gamma: Dict[ObjectType, float]
  eta: Dict[ObjectType, float]

  # does the human think the robot will succeed?
  def success_belief(self, theta, obj):

    g = self.gamma[obj]
    n = self.eta[obj]

    return sigmoid(g * theta + n)

  # what is the probability the human will stay put / not intervene?
  def prob_stay_put(self, theta, obj):

    b_tj = self.success_belief(theta, obj)
    rS = REWARD_TABLE[(obj, OutcomeType.SP_SUCCESS)]
    rF = REWARD_TABLE[(obj, OutcomeType.SP_FAIL)]

    return sigmoid(b_tj * rS + (1 - b_tj) * rF)

  def sample_human_action(self, theta, obj):

    p_stay = self.prob_stay_put(theta, obj)
    if np.random.rand() < p_stay:
      return None
    else: return OutcomeType.IT

In [7]:
human_model = HumanDecisionModel()

# bayesion inference for the probability of each observation
def belief(state_theta, action_obj, observation):

    p_stay = human_model.prob_stay_put(state_theta, action_obj)
    p_fail = ROBOT_FAIL_RATE[action_obj]

    if observation == OutcomeType.SP_SUCCESS:
        return p_stay * (1 - p_fail)

    elif observation == OutcomeType.SP_FAIL:
        return p_stay * p_fail

    else:
        return 1 - p_stay

In [8]:
from scipy.stats import norm

def belief_dist(theta_t, observation, theta_next=None, alpha=1.0, beta=0.2, epsilon=1.0, sigma=1):
    """
    P(θ_{t+1} | θ_t, a_t, o_t)

    - if robot succeeds:      trust increases by +alpha
    - if robot fails:         trust decreases by -beta
    - if human intervenes:    trust stays the same
    """
    if observation == OutcomeType.SP_SUCCESS:
        mean = theta_t + alpha
    elif observation == OutcomeType.SP_FAIL:
        mean = theta_t - epsilon
    else:
        mean = theta_t - beta

    probs = {}
    for theta in TRUST_LEVELS:
        lower = theta - 0.5
        upper = theta + 0.5
        probs[theta] = norm.cdf(upper, loc=mean, scale=sigma) - norm.cdf(lower, loc=mean, scale=sigma)

    total = sum(probs.values())
    probs = {theta: p / total for theta, p in probs.items()}

    if theta_next is not None:
        return probs[theta_next]
    else:
        return probs


In [9]:
def belief_update(belief, action, observation, human_model):

    predicted = {}
    for theta_next in TRUST_LEVELS:
        predicted[theta_next] = 0.0
        for theta_curr in TRUST_LEVELS:
            prior = belief[theta_curr]
            trans_prob = belief_dist(theta_curr, observation)[theta_next]
            predicted[theta_next] += prior * trans_prob

    new_belief = {}
    for theta_next in TRUST_LEVELS:

        p_stay = human_model.prob_stay_put(theta_next, action)
        p_fail = ROBOT_FAIL_RATE[action]
        p_succ = 1 - p_fail

        if observation == OutcomeType.SP_SUCCESS:
            obs_prob = p_stay * p_succ

        elif observation == OutcomeType.SP_FAIL:
            obs_prob = p_stay * p_fail

        else:
            obs_prob = 1 - p_stay

        new_belief[theta_next] = predicted[theta_next] * obs_prob

    # normalize
    total = sum(new_belief.values())
    new_belief = {theta: p / total for theta, p in new_belief.items()}

    return new_belief


In [10]:
from functools import lru_cache

# ChatGPT helped generate an exhaustive tree search in place of using SARSOP (which was incredibly difficult to download/run)
# The important distinction here is that, since our POMDP environment is so small, we are able to explore the entire state space
# Our results converge to a completely optimal policy (versus SARSOP uses approximation)
# This would NOT scale to a larger environment

def simple_trust_aware_policy(belief, human_model, available_actions):
    """
    Full-horizon trust-aware planning with memoization.
    Explores all trajectories until remaining_objects is empty.
    Returns (best_action, best_total_expected_reward).
    """

    init_belief_tuple = tuple(belief[theta] for theta in TRUST_LEVELS)
    init_actions_tuple = tuple(sorted([obj.value for obj in available_actions]))

    def immediate_expected_reward(belief_state, action_obj):
        expected = 0.0
        fail_rate = ROBOT_FAIL_RATE[action_obj]

        for theta in TRUST_LEVELS:
            p_theta = belief_state[theta]
            p_stay = human_model.prob_stay_put(theta, action_obj)

            r_success = REWARD_TABLE[(action_obj, OutcomeType.SP_SUCCESS)]
            r_fail    = REWARD_TABLE[(action_obj, OutcomeType.SP_FAIL)]
            r_it      = REWARD_TABLE[(action_obj, OutcomeType.IT)]

            # Outcome probabilities given theta, action
            p_IT   = 1 - p_stay
            p_fail = p_stay * fail_rate
            p_succ = p_stay * (1 - fail_rate)

            one_step = (
                p_succ * r_success +
                p_fail * r_fail +
                p_IT   * r_it
            )

            expected += p_theta * one_step

        return expected

    @lru_cache(maxsize=None)
    def cached_belief_update(belief_tuple, action_value, obs_value):
        belief_dict = {theta: belief_tuple[i] for i, theta in enumerate(TRUST_LEVELS)}
        action_obj = ObjectType(action_value)
        obs = OutcomeType(obs_value)
        new_b = belief_update(belief_dict, action_obj, obs, human_model)
        return tuple(new_b[theta] for theta in TRUST_LEVELS)

    @lru_cache(maxsize=None)
    def expected_return(belief_tuple, actions_tuple):
        if not actions_tuple:
            return (None, 0.0)

        belief_state = {theta: belief_tuple[i] for i, theta in enumerate(TRUST_LEVELS)}

        best_act = ObjectType(actions_tuple[0])
        best_val = -float('inf')

        for idx, action_value in enumerate(actions_tuple):
            action_obj = ObjectType(action_value)

            # One-step reward
            immediate = immediate_expected_reward(belief_state, action_obj)

            # Expected future reward
            future = 0.0
            for obs in (OutcomeType.SP_SUCCESS, OutcomeType.SP_FAIL, OutcomeType.IT):
                obs_value = obs.value

                if obs == OutcomeType.SP_SUCCESS:
                    p_obs = sum(
                        belief_state[t] *
                        (human_model.prob_stay_put(t, action_obj) *
                         (1 - ROBOT_FAIL_RATE[action_obj]))
                        for t in TRUST_LEVELS
                    )
                elif obs == OutcomeType.SP_FAIL:
                    p_obs = sum(
                        belief_state[t] *
                        (human_model.prob_stay_put(t, action_obj) *
                         ROBOT_FAIL_RATE[action_obj])
                        for t in TRUST_LEVELS
                    )
                elif obs == OutcomeType.IT:
                    p_obs = sum(
                        belief_state[t] *
                        (1 - human_model.prob_stay_put(t, action_obj))
                        for t in TRUST_LEVELS
                    )

                if p_obs == 0.0:
                    continue

                # Next belief (cached)
                next_b_tuple = cached_belief_update(belief_tuple, action_value, obs_value)

                # Next actions (remove current)
                next_actions = list(actions_tuple)
                next_actions.pop(idx)
                next_actions_tuple = tuple(sorted(next_actions))

                _, v_next = expected_return(next_b_tuple, next_actions_tuple)

                future += p_obs * v_next

            total = immediate + future

            if total > best_val:
                best_val = total
                best_act = action_obj

        return (best_act, best_val)

    return expected_return(init_belief_tuple, init_actions_tuple)


In [11]:
def simulate_task(human_model, initial_belief, initial_theta):
    """
    simulate a full table-clearing task
    """

    belief = initial_belief.copy()
    total_reward = 0.0
    history = []

    true_theta = initial_theta

    remaining_objects = [
        ObjectType.BOTTLE, ObjectType.BOTTLE, ObjectType.BOTTLE,
        ObjectType.CAN, ObjectType.GLASS
    ]

    while remaining_objects:

        action, _ = simple_trust_aware_policy(belief, human_model, remaining_objects)

        fail_rate = ROBOT_FAIL_RATE[action]
        rand = np.random.rand()

        if rand < fail_rate:
            observation = OutcomeType.SP_FAIL
        else:
            human_obs = human_model.sample_human_action(true_theta, action)
            if human_obs is None:
                observation = OutcomeType.SP_SUCCESS
            else:
                observation = OutcomeType.IT

        reward = REWARD_TABLE[(action, observation)]
        total_reward += reward

        trans_probs = belief_dist(true_theta, observation)
        true_theta = np.random.choice(
            TRUST_LEVELS,
            p=[trans_probs[theta] for theta in TRUST_LEVELS]
        )

        remaining_objects.remove(action)
        history.append(action.name)

    return total_reward, history


In [13]:
human_model = HumanDecisionModel()

human_model.gamma = {
    ObjectType.BOTTLE: 1, 
    ObjectType.CAN:    1.2,   
    ObjectType.GLASS:  1.5,  
}

human_model.eta = {
    ObjectType.BOTTLE: 0,  
    ObjectType.CAN:   -2.5,   
    ObjectType.GLASS: -5,   
}

initial_belief =   {
    1: 0.0625,
    2: 0.125,
    3: 0.1825,
    4: 0.375,
    5: 0.1825,
    6: 0.125,
    7: 0.0625
}

available = [
    ObjectType.BOTTLE, ObjectType.BOTTLE, ObjectType.BOTTLE,
    ObjectType.CAN, ObjectType.GLASS
]

total_reward, history = simulate_task(
            human_model,
            initial_belief,
            4
)

print("Total Reward: " + str(total_reward))
print("Action Sequence: " + str(history))

Total Reward: -10.0
Action Sequence: ['GLASS', 'BOTTLE', 'BOTTLE', 'BOTTLE', 'CAN']
